In [1]:
!pip install implicit

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/my_projects/retail_train.csv')
data.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [4]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

# Оценивание
За выполнени каждого задания 1 балл

4 балла -> отл

3 балла -> хор

И тд

### Задание 0. Товар 999999
На вебинаре мы использовали товар 999999 - что это за товар? Зачем он нужен. Используя этот товар мы смещяем качество рекомендаций. В какую сторону?  Можно ли удалить этот товар? Уберите этот товар и сравните с качеством на семинаре.

In [5]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

In [6]:
top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [7]:
data_train_top = data_train.loc[data_train['item_id'].isin(top_5000)]
data_train.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [8]:
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id',
                                  columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix[user_item_matrix > 0] = 1 # так как в итоге хотим предсказать 
user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

In [9]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [10]:
# %%time

model = ItemItemRecommender(K=5, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

recs = model.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, # кол-во рекомендаций 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=True)

In [11]:
recs

[(34707, 75614.0),
 (23590, 60381.0),
 (36454, 28271.0),
 (39619, 21721.0),
 (25064, 18600.0)]

In [12]:
result = pd.read_json('/content/drive/MyDrive/Colab Notebooks/my_projects/predictions_basic.json')
# result = result.loc[:, ['user_id', 'actual', 'itemitem']]
result.head()

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[9419193, 1001369, 1163749, 16807622, 1385008]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[999999, 1082185, 1029743, 995785, 1004906]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[881250, 981851, 1882086, 1033796, 1124210]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[999999, 1082185, 1098066, 6534178, 1127831]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[690271, 1527413, 950998, 10123377, 9884818]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 878996]","[999999, 1082185, 1029743, 6534178, 1127831]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[1095919, 2673691, 1067820, 724547, 392610]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 981760, 999999, 1127831, 961554]","[1082185, 981760, 1127831, 999999, 961554]","[999999, 1082185, 1029743, 1127831, 995785]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[1264779, 1583086, 5591140, 12649572, 6773055]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 1098066]","[1082185, 981760, 999999, 1098066, 826249]","[1082185, 981760, 999999, 1098066, 826249]","[999999, 1082185, 1029743, 1098066, 6534178]"


In [13]:
%%time

result['itemitem_only_5к'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in recs])

CPU times: user 6.53 ms, sys: 0 ns, total: 6.53 ms
Wall time: 7.28 ms


In [14]:
result.head()

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases,itemitem_only_5к
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[9419193, 1001369, 1163749, 16807622, 1385008]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[999999, 1082185, 1029743, 995785, 1004906]","[1082185, 981760, 1098066, 1127831, 995242]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[881250, 981851, 1882086, 1033796, 1124210]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[999999, 1082185, 1098066, 6534178, 1127831]","[1082185, 981760, 1098066, 1127831, 995242]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[690271, 1527413, 950998, 10123377, 9884818]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 878996]","[999999, 1082185, 1029743, 6534178, 1127831]","[1082185, 981760, 1098066, 1127831, 995242]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[1095919, 2673691, 1067820, 724547, 392610]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 981760, 999999, 1127831, 961554]","[1082185, 981760, 1127831, 999999, 961554]","[999999, 1082185, 1029743, 1127831, 995785]","[1082185, 981760, 1098066, 1127831, 995242]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[1264779, 1583086, 5591140, 12649572, 6773055]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 1098066]","[1082185, 981760, 999999, 1098066, 826249]","[1082185, 981760, 999999, 1098066, 826249]","[999999, 1082185, 1029743, 1098066, 6534178]","[1082185, 981760, 1098066, 1127831, 995242]"


In [15]:
def precision_at_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    flags = np.isin(bought_list, recommended_list)
    #print(flags)
    precision = flags.sum() / len(recommended_list)
    return precision

print('itemitem', round(result.apply(lambda row: precision_at_k(row['itemitem'], row['actual']), axis=1).mean(), 5))
print('itemitem_only_5к', round(result.apply(lambda row: precision_at_k(row['itemitem_only_5к'], row['actual']), axis=1).mean(), 5))

itemitem 0.13692
itemitem_only_5к 0.14574


Используя этот товар мы смещяем качество рекомендаций в худшую сторону.

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [16]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    items = items_weights['item_id'].values
    weights = items_weights['weight'].values

    recs = np.random.choice(items, size=n, replace=False, p=weights)

    # Подсказка: необходимо модифицировать функцию random_recommendation()
    # your_code
    
    return recs.tolist()

Сделайте предсказания

In [17]:
item_weights = data_train.groupby('item_id')['sales_value'].sum().reset_index()
item_weights['weight'] = item_weights['sales_value']/np.sum(item_weights['sales_value'])
item_weights.head()

,item_id,sales_value,weight
0,25671,20.94,2.969296e-06
1,26081,0.99,1.403822e-07
2,26093,1.59,2.254623e-07
3,26190,1.54,2.183723e-07
4,26355,1.98,2.807644e-07


In [18]:
%%time

result['weighted_random_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items_weights=item_weights, n=5))

CPU times: user 2.77 s, sys: 28.4 ms, total: 2.8 s
Wall time: 2.8 s


In [19]:
result.head()

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases,itemitem_only_5к,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[9419193, 1001369, 1163749, 16807622, 1385008]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[999999, 1082185, 1029743, 995785, 1004906]","[1082185, 981760, 1098066, 1127831, 995242]","[5995129, 5569845, 12352330, 9858652, 6553737]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[881250, 981851, 1882086, 1033796, 1124210]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[999999, 1082185, 1098066, 6534178, 1127831]","[1082185, 981760, 1098066, 1127831, 995242]","[1013794, 1081924, 1037000, 999250, 842140]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[690271, 1527413, 950998, 10123377, 9884818]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 878996]","[999999, 1082185, 1029743, 6534178, 1127831]","[1082185, 981760, 1098066, 1127831, 995242]","[1013321, 944018, 991874, 835300, 826580]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[1095919, 2673691, 1067820, 724547, 392610]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 981760, 999999, 1127831, 961554]","[1082185, 981760, 1127831, 999999, 961554]","[999999, 1082185, 1029743, 1127831, 995785]","[1082185, 981760, 1098066, 1127831, 995242]","[838602, 1041928, 1026910, 5591170, 1110031]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[1264779, 1583086, 5591140, 12649572, 6773055]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 1098066]","[1082185, 981760, 999999, 1098066, 826249]","[1082185, 981760, 999999, 1098066, 826249]","[999999, 1082185, 1029743, 1098066, 6534178]","[1082185, 981760, 1098066, 1127831, 995242]","[960253, 1082185, 8203686, 932631, 1089954]"


### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [20]:
# result = pd.read_csv('../predictions/predictions_basic.csv') # закгрузка predict с семианара
result.head()

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases,itemitem_only_5к,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[9419193, 1001369, 1163749, 16807622, 1385008]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[999999, 1082185, 1029743, 995785, 1004906]","[1082185, 981760, 1098066, 1127831, 995242]","[5995129, 5569845, 12352330, 9858652, 6553737]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[881250, 981851, 1882086, 1033796, 1124210]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[999999, 1082185, 1098066, 6534178, 1127831]","[1082185, 981760, 1098066, 1127831, 995242]","[1013794, 1081924, 1037000, 999250, 842140]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[690271, 1527413, 950998, 10123377, 9884818]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 878996]","[999999, 1082185, 1029743, 6534178, 1127831]","[1082185, 981760, 1098066, 1127831, 995242]","[1013321, 944018, 991874, 835300, 826580]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[1095919, 2673691, 1067820, 724547, 392610]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 981760, 999999, 1127831, 961554]","[1082185, 981760, 1127831, 999999, 961554]","[999999, 1082185, 1029743, 1127831, 995785]","[1082185, 981760, 1098066, 1127831, 995242]","[838602, 1041928, 1026910, 5591170, 1110031]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[1264779, 1583086, 5591140, 12649572, 6773055]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 1098066]","[1082185, 981760, 999999, 1098066, 826249]","[1082185, 981760, 999999, 1098066, 826249]","[999999, 1082185, 1029743, 1098066, 6534178]","[1082185, 981760, 1098066, 1127831, 995242]","[960253, 1082185, 8203686, 932631, 1089954]"


In [21]:
pd.DataFrame(
    [
        (model_, round(result.apply(lambda row: precision_at_k(recommended_list=row[str(model_)], bought_list=row['actual']), axis=1).mean(), 5)) for model_ in list(result)[2:]
    ], 
    columns=['metric', 'value']
)

,metric,value
0,random_recommendation,0.00078
1,popular_recommendation,0.15524
2,itemitem,0.13692
3,cosine,0.13291
4,tfidf,0.13898
5,own_purchases,0.17969
6,itemitem_only_5к,0.14574
7,weighted_random_recommendation,0.02194


### Задание 3. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.
-  *Попробуйте стратегии ансамблирования изученных алгоритмов

Обязательно нужно сделать первые 2 пункта!

In [22]:
def random_recommendation(items, n=5):
    """Случайные рекоммендации"""
    
    
    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()

In [23]:
def popularity_recommendation(data, n=5):
    """Топ-n популярных товаров"""
    
    popular = data.groupby('item_id')['sales_value'].sum().reset_index()
    popular.sort_values('sales_value', ascending=False, inplace=True)
    
    recs = popular.head(n).item_id
    
    return recs.tolist()

In [24]:
data_train2 = data_train.loc[data_train['item_id'].isin(top_5000),]

In [25]:
%%time

items = data_train.item_id.unique()
result['random_recommendation'] = result['user_id'].apply(lambda x: random_recommendation(items, n=5))

CPU times: user 3.85 s, sys: 37.9 ms, total: 3.89 s
Wall time: 3.92 s


In [26]:
%%time

items = data_train2.item_id.unique()
result['random_recommendation_2'] = result['user_id'].apply(lambda x: random_recommendation(items, n=5))

CPU times: user 248 ms, sys: 1.06 ms, total: 249 ms
Wall time: 251 ms


In [27]:
%%time

popular_recs = popularity_recommendation(data_train, n=5)
result['popular_recommendation'] = result['user_id'].apply(lambda x: popular_recs)

CPU times: user 138 ms, sys: 2.16 ms, total: 140 ms
Wall time: 143 ms


In [28]:
%%time

popular_recs = popularity_recommendation(data_train2, n=5)
result['popular_recommendation_2'] = result['user_id'].apply(lambda x: popular_recs)

CPU times: user 46.6 ms, sys: 2.13 ms, total: 48.8 ms
Wall time: 50.4 ms


In [31]:
%%time

result['weighted_random_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items_weights=item_weights, n=5))
result.head()

CPU times: user 2.81 s, sys: 31.8 ms, total: 2.84 s
Wall time: 2.85 s


In [32]:
pd.DataFrame([
    result.apply(lambda row: precision_at_k(row['random_recommendation'], row['actual']), axis=1).mean(),
    result.apply(lambda row: precision_at_k(row['random_recommendation_2'], row['actual']), axis=1).mean(),
    result.apply(lambda row: precision_at_k(row['popular_recommendation'], row['actual']), axis=1).mean(),
    result.apply(lambda row: precision_at_k(row['popular_recommendation_2'], row['actual']), axis=1).mean(),
], 
    columns=['precision_at_k'], 
    index=['random_recommendation', 'random_recommendation_5к', 'popular_recommendation', 'popular_recommendation_5к']
).T

,random_recommendation,random_recommendation_5к,popular_recommendation,popular_recommendation_5к
precision_at_k,0.000588,0.006758,0.15524,0.15524


In [33]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns = ['user_id', 'actual']

In [34]:
%%time

for k in range(1, 16):
    
    model = ItemItemRecommender(K=k, num_threads=4)  # K - кол-во билжайших соседей
    model.fit(csr_matrix(user_item_matrix).T.tocsr(), show_progress=True)
    
    recs = model.recommend(
        userid=userid_to_id[2],  # userid - id от 0 до N
        user_items=csr_matrix(user_item_matrix).tocsr(),  # на вход user-item matrix
        N=5,  # кол-во рекомендаций 
        filter_already_liked_items=False,
        filter_items=None,
        recalculate_user=True)
    
    result[f'itemitem_K={k}'] = result['user_id'].\
        apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                        model.recommend(userid=userid_to_id[x], 
                                        user_items=sparse_user_item,   # на вход user-item matrix
                                        N=5, 
                                        filter_already_liked_items=False, 
                                        filter_items=None, 
                                        recalculate_user=True)])


CPU times: user 7min 38s, sys: 2.81 s, total: 7min 41s
Wall time: 5min 48s


In [35]:
pd.DataFrame(
    [
        result.apply(lambda row: precision_at_k(row['itemitem_K=1'], row['actual']), axis=1).mean(),
        result.apply(lambda row: precision_at_k(row['itemitem_K=2'], row['actual']), axis=1).mean(),
        result.apply(lambda row: precision_at_k(row['itemitem_K=3'], row['actual']), axis=1).mean(),
        result.apply(lambda row: precision_at_k(row['itemitem_K=4'], row['actual']), axis=1).mean(),
        result.apply(lambda row: precision_at_k(row['itemitem_K=5'], row['actual']), axis=1).mean(),
        result.apply(lambda row: precision_at_k(row['itemitem_K=6'], row['actual']), axis=1).mean(),
        result.apply(lambda row: precision_at_k(row['itemitem_K=7'], row['actual']), axis=1).mean(),
        result.apply(lambda row: precision_at_k(row['itemitem_K=8'], row['actual']), axis=1).mean(),
        result.apply(lambda row: precision_at_k(row['itemitem_K=9'], row['actual']), axis=1).mean(),
        result.apply(lambda row: precision_at_k(row['itemitem_K=10'], row['actual']), axis=1).mean(),
        result.apply(lambda row: precision_at_k(row['itemitem_K=11'], row['actual']), axis=1).mean(),
        result.apply(lambda row: precision_at_k(row['itemitem_K=12'], row['actual']), axis=1).mean(),
        result.apply(lambda row: precision_at_k(row['itemitem_K=13'], row['actual']), axis=1).mean(),
        result.apply(lambda row: precision_at_k(row['itemitem_K=14'], row['actual']), axis=1).mean(),
        result.apply(lambda row: precision_at_k(row['itemitem_K=15'], row['actual']), axis=1).mean(),
    ], columns = ['precision_at_k'], index=[f'k={i}' for i in range(1, 16)]).sort_values(by='precision_at_k', ascending=False).T

,k=1,k=2,k=3,k=15,k=10,k=11,k=9,k=14,k=12,k=13,k=8,k=7,k=6,k=5,k=4
precision_at_k,0.219491,0.206268,0.170029,0.167091,0.166993,0.166993,0.166895,0.166895,0.166601,0.166308,0.164643,0.164153,0.15906,0.154065,0.151028


### Задание 4. Улучшение детерминированных алгоритмов
На семинаре мы рассматривали 



Далее $U \equiv N_i(u) $

$$r_{u,i} =  \frac{1}{S}\sum\limits_{v \in U}\operatorname{sim}(u,v)r_{v, i}$$
$$ S = \sum\limits_{v \in U} \operatorname{sim}(u,v)$$

Предлагается улучшить эту формулу и учесть средние предпочтения всех пользователей

$$r_{u,i} = \mu + \bar{r_u} + \frac{1}{S}\sum\limits_{v \in U}\operatorname{sim}(u,v)(r_{v, i}-\bar{r_{v}} - \mu)$$

Какие смысл имееют $ \mu $ и $ \bar{r_u}$ ?

Реализуйте алгоритм, прогнозирующий рейтинги на основе данной формулы, на numpy (векторизованно!)

В качестве схожести возьмите CosineSimilarity.

Примените к user_item_matrix. В качестве рейтингов возьмите количество или стоимость купленного товара. 
Данный алгоритм предсказывает рейтинги. Как на основании предсказанных рейтингов предсказать факт покупки?

Предложите вариант.
Посчитайте accuracy@5 и сравните с алгоритмами, разобранными на вебинаре.

$ \mu $ - средняя оценка по всей таблице  
$ \bar{r_u}$ - средний рейтинг товаров, которые оценил пользователь  